In [ ]:
from power_planner.utils.utils import get_donut_vals, get_half_donut, get_distance_surface, bresenham_line, rescale, get_lg_donut, angle, normalize
from power_planner.plotting import plot_path, plot_path_costs
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import time

In [ ]:
from power_planner import graphs

In [ ]:
instance_path = "../data/data_dump_5.dat"
with open(instance_path, "rb") as infile:
    data = pickle.load(infile)
(instance, instance_corr, start_inds, dest_inds) = data.data

In [ ]:
min_dist = 3
max_dist = 5

In [ ]:
class TwoPowerBF():
    
    def __init__(self,
        instance,
        instance_corr,
        graphtool=1,
        directed=True,
        verbose=1,
        n_iters=50,
        fill_val=np.inf
    ):
        self.graph_ab = graphs.PowerBF(instance, instance_corr, graphtool=1, verbose=1)
        self.graph_ba = graphs.PowerBF(instance, instance_corr, graphtool=1, verbose=1)

    def set_edge_costs(self, layer_classes, class_weights, angle_weight=0.5):
        self.graph_ab.set_edge_costs(layer_classes, class_weights, angle_weight=angle_weight)
        self.graph_ba.set_edge_costs(layer_classes, class_weights, angle_weight=angle_weight)

    def set_shift(self, min_dist, max_dist, vec, max_angle, max_angle_lg):
        self.graph_ab.set_shift(min_dist, max_dist, vec, max_angle, max_angle_lg)
        self.graph_ba.set_shift(min_dist, max_dist, np.asarray(vec)*(-1), max_angle, max_angle_lg)
        
    def add_nodes(self):
        self.graph_ab.add_nodes()
        self.graph_ba.add_nodes()

    def set_corridor(self, factor, corridor, start_inds, dest_inds):
        self.graph_ab.set_corridor(factor, corridor, start_inds, dest_inds)
        self.graph_ba.set_corridor(factor, corridor, dest_inds, start_inds)
        
    def add_edges(self):
        self.graph_ab.add_edges()
        self.graph_ba.add_edges()

    def get_shortest_path(self, start_inds, dest_inds):
        self.path_ab, _, _ = self.graph_ab.get_shortest_path(start_inds, dest_inds)
        self.path_ba, _, _ = self.graph_ba.get_shortest_path(dest_inds, start_inds)
        assert np.all(np.flip(np.asarray(self.path_ba), axis=0)==self.path_ab)

    def best_in_window(self, w_xmin, w_xmax, w_ymin, w_ymax, start_ind, dest_inds, margin=0.05):
        """
        margin: percent that it's allowed to be higher than average
        """
        opt = np.min(self.graph_ab.dists[:, dest_inds[0], dest_inds[1]])
                
        possible_cs = []
        for x in range(w_xmin, w_xmax+1, 1):
            for y in range(w_ymin, w_ymax+1, 1):
                # todo here: take into account angle directly
                added_costs = np.min(self.graph_ab.dists[:, x, y]) +  np.min(self.graph_ba.dists[:, x, y]) - self.graph_ab.instance[x,y]
                if added_costs < opt + margin * opt:
                    possible_cs.append(np.array([x,y]))
        for c in possible_cs:
            path_ac = self.graph_ab.get_shortest_path(start_inds, c, ret_only_path=True)
            path_cb = self.graph_ba.get_shortest_path(dest_inds, c, ret_only_path=True)
            plt.plot(path_ac[:,0], path_ac[:,1])
            plt.plot(path_cb[:,0], path_cb[:,1])
            plt.show()

In [ ]:
graph = TwoPowerBF(
    instance, instance_corr, graphtool=1, verbose=1
)

graph.set_edge_costs(
    data.layer_classes, data.class_weights, angle_weight=0.1
)
graph.set_shift(min_dist, max_dist, dest_inds-start_inds, 0.5*np.pi, 0.25*np.pi)

# add vertices
graph.add_nodes()

graph.set_corridor(1, None, start_inds, dest_inds)
print("1) set cost rest")

graph.add_edges()


In [ ]:
# get actual best path
graph.get_shortest_path(start_inds, dest_inds)
print("3) shortest path")

In [ ]:
def get_sp(dists, dists_argmin, start_inds, dest_inds, shifts, min_shift):
    if not np.any(dists[:, dest_inds[0], dest_inds[1]] < np.inf):
        raise RuntimeWarning("empty path")
    curr_point = dest_inds
    my_path = [dest_inds]
    # min_shift = np.argmin(dists[:, dest_inds[0], dest_inds[1]])
    while np.any(curr_point - start_inds):
        new_point = curr_point - shifts[int(min_shift)]
        min_shift = dists_argmin[int(min_shift), curr_point[0], curr_point[1]]
        my_path.append(new_point)
        curr_point = new_point
    return np.asarray(my_path)

def best_in_window(two_power_obj, w_xmin, w_xmax, w_ymin, w_ymax, start_ind, dest_inds, margin=0.05):
    """
    margin: percent that it's allowed to be higher than average
    """
    opt = np.min(two_power_obj.graph_ab.dists[:, dest_inds[0], dest_inds[1]])

    ang_weight = two_power_obj.graph_ba.angle_weight
    ang_norm_factor = two_power_obj.graph_ba.angle_norm_factor
            
    possible_cs = []
    c_path_cost = []
    possible_shifts = []
            
    for x in range(w_xmin, w_xmax+1, 1):
        for y in range(w_ymin, w_ymax+1, 1):
            # todo here: take into account angle directly
            cell_val = two_power_obj.graph_ab.instance[x,y]
            if cell_val < np.inf:
                min_costs = np.inf
                min_shifts = [0,0]
                for s1 in range(len(two_power_obj.graph_ab.shifts)):
                    for s2 in range(len(two_power_obj.graph_ab.shifts)):
                        val_ab = two_power_obj.graph_ab.dists[s1, x, y]
                        shift_ab = two_power_obj.graph_ab.shifts[s1]
                        val_ba = two_power_obj.graph_ba.dists[s2, x, y]
                        shift_ba = two_power_obj.graph_ba.shifts[s2]
                        ang = angle(np.asarray(shift_ab), np.asarray(shift_ba)*(-1))
                        added_costs = val_ab + val_ba - cell_val + ang_weight * ang / ang_norm_factor
                        if added_costs < min_costs:
                            min_costs = added_costs
                            min_shifts = [s1, s2]
                possible_shifts.append(min_shifts)
                added_costs = min_costs # np.min(two_power_obj.graph_ab.dists[:, x, y]) +  np.min(two_power_obj.graph_ba.dists[:, x, y]) - two_power_obj.graph_ab.instance[x,y]
            else:
                possible_shifts.append([0,0])
                added_costs = np.inf
            possible_cs.append(np.array([x,y]))
            c_path_cost.append(min_costs)
            
    # get best one
    best_c = np.argmin(c_path_cost)
    print(best_c)
    c = possible_cs[best_c]
    s1, s2 = possible_shifts[best_c]
    # stick together the path
    path_ac = get_sp(two_power_obj.graph_ab.dists, two_power_obj.graph_ab.dists_argmin, start_inds, c, two_power_obj.graph_ab.shifts, s1)
    path_cb = get_sp(two_power_obj.graph_ba.dists, two_power_obj.graph_ba.dists_argmin, dest_inds, c, two_power_obj.graph_ba.shifts, s2)
    # path_ac = two_power_obj.graph_ab.get_shortest_path(start_inds, c, ret_only_path=True)
    # path_cb = two_power_obj.graph_ba.get_shortest_path(dest_inds, c, ret_only_path=True)
    # return concatenated path and new costs
    print(opt, c_path_cost[best_c], c)
    together = np.concatenate((np.flip(np.array(path_ac), axis=0), np.array(path_cb)), axis=0)
    plt.plot(path_ac[:,0], path_ac[:,1])
    plt.plot(path_cb[:,0], path_cb[:,1])
    plt.show()
    return together
    

In [ ]:
new_path = best_in_window(graph, 70,75, 185, 190, start_inds, dest_inds, margin = 0.2)

In [ ]:
path = np.array(graph.path_ab)
plt.figure(figsize=(20,10))
plt.imshow(np.swapaxes(graph.graph_ab.instance,1,0))
plt.plot(path[:,0], path[:,1],color="yellow")
plt.plot(new_path[:,0], new_path[:,1],color="orange")
plt.show()

In [ ]:
class BFangleKSP():
    
    def __init__(self, bf_graph):
        self.graph = bf_graph
        self.dists_ab = bf_graph.dists.copy()
        self.dists_argmin_ab = bf_graph.dists_argmin.copy()
        self.path_ab = bf_graph.best_path
        
    def add_ba(self):
        graph.shifts = np.array(self.graph.shifts) * (-1)
        # set dists to zero
        self.graph.add_nodes()
        # set dists of dest_inds
        self.graph.set_corridor(1, None, dest_inds, start_inds)
        # compute dists array
        self.graph.add_edges()
        # save new results
        self.dists_ba = self.graph.dists
        self.dists_argmin_ba = self.graph.dists_argmin
        self.path_ba, _, _ = self.graph.get_shortest_path(dest_inds, start_inds)
        assert np.all(np.flip(np.asarray(self.path_ba), axis=0)==self.path_ab)

    

    # best_in_window(45, 48, 60, 63, dists_ab, dists_ba, dest_inds, test_example)
        
        
    def on_path(self, point, best_path):
        point = np.array(point)
        return any([not np.any(point-p) for p in best_path])
        # test:
        # print(on_path(best_path[20], best_path), on_path(best_path[20]+1, best_path))
        
           
    @staticmethod
    def get_sp(dists, dists_argmin, start_inds, dest_inds, shifts):
        if not np.any(dists[:, dest_inds[0], dest_inds[1]] < np.inf):
            raise RuntimeWarning("empty path")
        curr_point = dest_inds
        my_path = [dest_inds]
        min_shift = np.argmin(dists[:, dest_inds[0], dest_inds[1]])
        while np.any(curr_point - start_inds):
            new_point = curr_point - shifts[int(min_shift)]
            min_shift = dists_argmin[int(min_shift), curr_point[0], curr_point[1]]
            my_path.append(new_point)
            curr_point = new_point
        return np.asarray(my_path)

In [ ]:
bf_ang = BFangleKSP(graph)

In [ ]:
bf_ang.add_ba()

In [ ]:
bf_ang.best_in_window(80,90, 140, 150, start_inds, dest_inds)

In [ ]:
g = Graph()

In [ ]:
g.add_vertex(20)

In [ ]:
for e in range(5,15):
    print(e,e+1)
    g.add_edge(e,e+1)

In [ ]:
dists,preds = shortest_distance(g, 10, pred_map=True) # dist_map mitgeben wird nicht überschrieben!
# dists of a nonexistant vertex is 0!

In [ ]:
dists[5]

In [ ]:
g.set_reversed(is_reversed=True)

## with graph-tool

In [ ]:
from graph_tool.all import *

In [ ]:
graph = Graph(directed=True)
graph.add_vertex(40)
for i in range(20): # np.random.randint(0,40,50):
    second = np.random.randint(0,40,10)
    for s in second:
        graph.add_edge(i,s)

In [ ]:
[v for v in graph.vertex(5).out_edges()]

In [ ]:
source = 5
target=3

In [ ]:
vertices, _ = shortest_path(
                graph,
                source,
                target
            )

In [ ]:
vertices

In [ ]:
3-18-5  5-2-3

In [ ]:
dist_map_ab, pred_map_ab = shortest_distance(
                graph,
                source,
                pred_map=True
            )

In [ ]:
dist_map_ab[target], pred_map_ab[target]

In [ ]:
graph.set_reversed(is_reversed=True)

In [ ]:
dist_map_ba, pred_map_ba = shortest_distance(
        graph,
        target,
    pred_map = True
    )

In [ ]:
dist_map_ba[source], pred_map_ba[source]

In [ ]:
def get_sp_from_preds(pred_map, curr_vertex, start_vertex):
    path = [curr_vertex]
    while curr_vertex != start_vertex:
        curr_vertex = pred_map[curr_vertex]
        path.append(curr_vertex)
    return path

In [ ]:
get_sp_from_preds(pred_map_ba, source, target)

In [ ]:
shifts = get_half_donut(3,5,[1,0])
np.asarray(shifts)

In [ ]:
get_sp_from_preds(pred_map_ab, target, source)

In [ ]:
a = [[1,2],[3,4]]
a.reverse()

In [ ]:
a

In [ ]:
def best_in_window(
        graph,
        w_xmin,
        w_xmax,
        w_ymin,
        w_ymax,
        source,
        dest,
        margin=0.05
    ):
    for x in range(w_xmin, w_xmax + 1, 1):
        for y in range(w_ymin, w_ymax + 1, 1):
            v = pos2node[x,y]
            # get dists_map_ab[v]
            # get_sp_from_preds(pred_map_ba, source, target)

In [ ]:
path = "../../outputs/scenarios/path_scenario"

plt.figure(figsize=(20,10))
for i, img_name in enumerate(["1.png", "2.png", "3.png"]):
    img = plt.imread(path+img_name)
    plt.subplot(1,3,i+1)
    plt.imshow(img[40:, 25:])
    plt.axis("off")
    plt.title("scenario "+img_name[0], fontsize=20)
plt.savefig(path+".pdf")
plt.show()

In [ ]:
import json
with open("../../outputs/path_kshortest_ksp.json", "r") as outfile:
    ksp = json.load(outfile)
    

In [ ]:
ksp[0]

In [ ]:
costs = [k[2] for k in ksp]
print(costs)

In [ ]:
paths = [k[0] for k in ksp]

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(np.swapaxes(np.mean(instance, axis=0), 1, 0))
# plt.imshow(np.swapaxes(instance[1:], 2,0), vmin=0, vmax=0.2)
for i, path in enumerate(paths):
    path = np.asarray(path)
    plt.plot(path[:,0], path[:,1], label=str(round(costs[i], 2)), linewidth=3)

leg = plt.legend(fontsize=15)
leg.set_title('Costs',prop={'size':15})
# plt.legend(title="costs", fontsize=15, )
plt.axis("off")
plt.savefig("k_shortest.pdf")
plt.show()

In [ ]:
 def k_shortest_paths(self, source, dest, k, overlap=0.5, mode="myset"):
        tic = time.time()
        # initialize list of paths
        sp_set = set(self.best_path)
        best_paths = [self.best_path]
        best_path_sets = [set(self.best_path)]
        # get list of vertices = unique values in pos2node except -1
        vertices = np.unique(self.pos2node)[1:]
        v_dists = [self.dist_map_ab[v] + self.dist_map_ba[v] for v in vertices]
        # sort paths
        v_shortest = np.argsort(v_dists)
        # iterate over vertices starting from shortest paths
        # times_getpath = []
        for j, v_ind in enumerate(v_shortest):
            v = vertices[v_ind]
            # TODO: for runtime scan only every xth one (anyways diverse)
            if v not in sp_set:
                # do not scan unreachable vertices
                if int(self.pred_map_ab[v]
                       ) == int(v) or int(self.pred_map_ba[v]) == int(v):
                    continue
                # tic1 = time.time()
                try:
                    path_ac = self.get_sp_from_preds(
                        self.pred_map_ab, v, source
                    )
                    path_cb = self.get_sp_from_preds(self.pred_map_ba, v, dest)
                except RuntimeWarning:
                    print("while loop not terminating")
                    continue
                # times_getpath.append(time.time() - tic1)
                path_ac.reverse()
                # concatenate - leave 1 away because otherwise twice
                vertices_path = path_ac + path_cb[1:]

                # similar = similarity(vertices_path, best_paths, sp_set)
                if mode != "myset":
                    sofar = np.array(
                        [
                            WeightedKSP.similarity(
                                sp, set(vertices_path), mode
                            ) for sp in best_path_sets
                        ]
                    )
                    if np.all(sofar < overlap):
                        best_paths.append(vertices_path)
                        best_path_sets.append(set(vertices_path))
                # mode myset --> my version: set of all paths together
                else:
                    already = np.array([u in sp_set for u in vertices_path])
                    if np.sum(already) < len(already) * overlap:
                        best_paths.append(vertices_path)
                        sp_set.update(vertices_path)
                    # print("added path, already scanned", j)
            # stop if k paths are sampled
            if len(best_paths) >= k:
                break

        self.time_logs["ksp"] = round(time.time() - tic, 3)
        return [self.transform_path(p) for p in best_paths]